<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/kurs-analiza-danych-2022/blob/main/Tydzie%C5%84%206/Warsztaty_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Sparka

## Utworzenie środowiska pyspark do obliczeń

Tworzymy swoje środowisko z pysparkiem we wenętrzu naszych zasobów chmurowych

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q ftp.ps.pl/pub/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [5]:
!pip install -q findspark

import findspark
findspark.init()

## Utworzenie sesji z pyspark


Utworzymy testowo sesję aby zobaczyć czy działa. Element ten jest wspólny również gdy systemy sparkowe pracują w sposób ciągły, a nie są tworzone przez naszą sesję.

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

## Podłączenie Google Drive do sesji colab

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
pola_zbiorczo = '''YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY'''
pola = pola_zbiorczo.split(',')

## Przygotowanie Dataframe z pełnym schematem danych

Poprzednio poszliśmy na skróty przypisując każdej ze zmiennych typ ciągu znaków. Tym razem zróbmy to porządnie

In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [10]:
from pyspark.sql.types import StructType, StringType, IntegerType, BooleanType, FloatType, TimestampType, DateType, ArrayType, MapType
from typing import List, Tuple, Dict, Any
map_python_types_2_spark_types = {
    str : StringType(),
    int : IntegerType(),
    bool : BooleanType(),
    float: FloatType(),
    'timestamp' : TimestampType(),
    'date' : DateType(),
    List[str] : ArrayType(StringType()),
    Tuple[str] : ArrayType(StringType()),
    Dict[str, str] : MapType(StringType(), StringType())
}

column_type_collection = {
    int : ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_DELAY', 'TAXI_OUT', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'TAXI_IN', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED' ],
    str : ['AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF', 'WHEELS_ON', 
      'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
    ]
}

map_column_names_2_types = {}

for pole in pola:
  for python_type, column_list in column_type_collection.items():
    if pole in column_list:
      map_column_names_2_types[pole] = map_python_types_2_spark_types[python_type]

print(map_column_names_2_types)



{'YEAR': IntegerType, 'MONTH': IntegerType, 'DAY': IntegerType, 'DAY_OF_WEEK': IntegerType, 'AIRLINE': StringType, 'FLIGHT_NUMBER': StringType, 'TAIL_NUMBER': StringType, 'ORIGIN_AIRPORT': StringType, 'DESTINATION_AIRPORT': StringType, 'SCHEDULED_DEPARTURE': StringType, 'DEPARTURE_TIME': StringType, 'DEPARTURE_DELAY': IntegerType, 'TAXI_OUT': IntegerType, 'WHEELS_OFF': StringType, 'SCHEDULED_TIME': IntegerType, 'ELAPSED_TIME': IntegerType, 'AIR_TIME': IntegerType, 'DISTANCE': IntegerType, 'WHEELS_ON': StringType, 'TAXI_IN': IntegerType, 'SCHEDULED_ARRIVAL': StringType, 'ARRIVAL_TIME': StringType, 'ARRIVAL_DELAY': IntegerType, 'DIVERTED': IntegerType, 'CANCELLED': IntegerType, 'CANCELLATION_REASON': StringType, 'AIR_SYSTEM_DELAY': StringType, 'SECURITY_DELAY': StringType, 'AIRLINE_DELAY': StringType, 'LATE_AIRCRAFT_DELAY': StringType, 'WEATHER_DELAY': StringType}


In [11]:
schemat = StructType()
for pole, typ in map_column_names_2_types.items():
    schemat = schemat.add(pole, typ, True)


In [12]:
flights = spark.read.format('csv').option("header", False).schema(schemat).load('/content/drive/MyDrive/flights.csv')
flights.show(5)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [13]:
flights.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = 

In [14]:
flights.select(flights.DAY).distinct().show()

+---+
|DAY|
+---+
| 31|
| 28|
| 26|
| 27|
| 12|
| 22|
|  1|
| 13|
|  6|
| 16|
|  3|
| 20|
|  5|
| 19|
| 15|
|  9|
| 17|
|  4|
|  8|
| 23|
+---+
only showing top 20 rows



Wykonajmy jeszcze proste statystyki ze zbioru by zobaczyć poprawność jego wczytania

In [15]:
%%time
flights.count()

CPU times: user 22.1 ms, sys: 7.23 ms, total: 29.3 ms
Wall time: 4.21 s


5819079

In [16]:
%%time
for pole in ['YEAR', 'MONTH', 'DAY', 'AIRLINE', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DIVERTED', 'CANCELLED']:
  print(f'Pole {pole}\n')
  print(flights.select(pole).distinct().sort(pole).toPandas())

Pole YEAR

   YEAR
0  2015
Pole MONTH

    MONTH
0       1
1       2
2       3
3       4
4       5
5       6
6       7
7       8
8       9
9      10
10     11
11     12
Pole DAY

    DAY
0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
10   11
11   12
12   13
13   14
14   15
15   16
16   17
17   18
18   19
19   20
20   21
21   22
22   23
23   24
24   25
25   26
26   27
27   28
28   29
29   30
30   31
Pole AIRLINE

   AIRLINE
0       AA
1       AS
2       B6
3       DL
4       EV
5       F9
6       HA
7       MQ
8       NK
9       OO
10      UA
11      US
12      VX
13      WN
Pole TAIL_NUMBER

     TAIL_NUMBER
0           None
1          7819A
2          7820L
3         D942DN
4         N001AA
...          ...
4893      N997DL
4894      N998AT
4895      N998DL
4896      N999DN
4897      N9EAMQ

[4898 rows x 1 columns]
Pole ORIGIN_AIRPORT

    ORIGIN_AIRPORT
0            10135
1            10136
2            10140
3            10141
4            10146
..  

In [17]:
flights.createOrReplaceTempView("flights")

## Ćwiczenia 

Zadanie w ćwiczeniu polegać będą na stworzeniu konwersji zapytania Spark-SQL na składnię pyspark

In [18]:
spark.sql('select count(*) from flights where cancelled = 1').toPandas()

,count(1)
0,89884


In [19]:
flights.where("cancelled = 1").count()

89884

In [ ]:
spark.sql('select count(*) from flights where diverted = 1').toPandas()

,count(1)
0,15187


In [20]:
flights.where("diverted =1").count()

15187

In [22]:
%%time
spark.sql('select count(*) from flights where cancelled = 1 and diverted = 1').toPandas()

CPU times: user 60.8 ms, sys: 5.72 ms, total: 66.5 ms
Wall time: 10.1 s


,count(1)
0,0


In [23]:
%%time
flights.where("cancelled =1").where("diverted =1").count()

CPU times: user 51 ms, sys: 3.29 ms, total: 54.3 ms
Wall time: 9.57 s


0

In [24]:
%%time
flights.where("cancelled =1 and diverted =1").count()

CPU times: user 46.5 ms, sys: 8.48 ms, total: 54.9 ms
Wall time: 9.59 s


0

In [25]:
%%time
flights.where("diverted =1").where("cancelled =1").count()

CPU times: user 51.9 ms, sys: 5.89 ms, total: 57.7 ms
Wall time: 9.56 s


0

In [ ]:
spark.sql('select avg(distance) from flights').toPandas()

,avg(distance)
0,822.356495


In [26]:
from pyspark.sql import functions as sf

flights.agg(sf.avg('distance')).show()

+-----------------+
|    avg(distance)|
+-----------------+
|822.3564947305235|
+-----------------+



In [ ]:
spark.sql('select min(DEPARTURE_DELAY), max(DEPARTURE_DELAY) from flights').toPandas()

,min(DEPARTURE_DELAY),max(DEPARTURE_DELAY)
0,-82,1988


In [27]:
flights.agg(sf.min('DEPARTURE_DELAY'),sf.max('DEPARTURE_DELAY')).show()


+--------------------+--------------------+
|min(DEPARTURE_DELAY)|max(DEPARTURE_DELAY)|
+--------------------+--------------------+
|                 -82|                1988|
+--------------------+--------------------+



In [ ]:
spark.sql('select min(ARRIVAL_DELAY), max(ARRIVAL_DELAY) from flights').toPandas()

,min(ARRIVAL_DELAY),max(ARRIVAL_DELAY)
0,-87,1971


# Ćwiczenie warsztatowe

## Zadanie 1 

Utworzyć podzbiór zawierający tylko loty, które się odbyły. Znaleźć lot najkrótszy oraz najdłuższy.

## Zadanie 2

Wyszukać liczbę przewozników w danych i znaleźć łączną liczbę lotów wykonanych przez każdego z nich

## Zadanie 3

Dla każdej trasy (Lotnisko początkowe -> Lotnisko końcowe) znaleźć minimalny, przeciętny i maksymalny (rzeczywisty) czas przelotu

## Zadanie 4

Utworzyć nową kolumnę opisującą trasę (Lotnisko początkowe -> Lotnisko końcowe). Następnie sprawdzić czy w danych podanych jest zgodna odległość jego łączące

## Zadanie 5

Wygenerować tabelę z popularnością poszczególnych przewoźników na podstawie całego zbioru danych, wygenerować ich ranking i dołączyć go do danych flights jako kolumnę AIRLINE_RANK.


In [29]:
#zadanie 1 Utworzyć podzbiór zawierający tylko loty, które się odbyły. Znaleźć lot najkrótszy oraz najdłuższy.
flights_real = flights.where('cancelled = 0')

flights_real.agg(sf.min('DISTANCE'), sf.max('DISTANCE')).show()


+-------------+-------------+
|min(DISTANCE)|max(DISTANCE)|
+-------------+-------------+
|           31|         4983|
+-------------+-------------+



In [30]:
#Zadanie 2
# Wyszukać liczbę przewozników w danych i znaleźć łączną liczbę lotów wykonanych przez każdego z nich

flights_real.groupBy("Airline").count().show()

+-------+-------+
|Airline|  count|
+-------+-------+
|     UA| 509150|
|     NK| 115375|
|     AA| 715065|
|     EV| 556746|
|     B6| 262772|
|     DL| 872057|
|     OO| 578393|
|     F9|  90248|
|     US| 194648|
|     MQ| 279607|
|     HA|  76101|
|     AS| 171852|
|     VX|  61369|
|     WN|1245812|
+-------+-------+



In [31]:
flights_real.groupBy("Airline").count().count()

14

In [34]:
flights_real.select('airline').distinct().count()

14

In [33]:
from pyspark.sql.functions import col
a_airlines=flights_real.groupBy('AIRLINE').count().orderBy(col('count').desc())

a_airlines.toPandas()

,AIRLINE,count
0,WN,1245812
1,DL,872057
2,AA,715065
3,OO,578393
4,EV,556746
5,UA,509150
6,MQ,279607
7,B6,262772
8,US,194648
9,AS,171852
